<a href="https://colab.research.google.com/github/akritia3/gradle-chatbot/blob/main/gradle_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Generative AI · Code Ada Day 1
### Workshop led by Illinois WCS Explorations, October 14 2023

To get started, make a copy of this notebook: File → Save a copy in Drive

Sources:
- [Gradio Interface](https://www.gradio.app/guides/interface-state) documentation
- [Hugging Face Transformers](https://huggingface.co/docs/transformers/v4.34.0/en/llm_tutorial) documentation

## Install and import libraries

Warning: Installation of libraries may take a while.

In [ ]:
!pip install transformers
!pip install gradio
!pip install torch

from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.8 MB/s e

## Set up the basics

Instantiate the tokenizer and model using `AutoTokenizer` and `AutoModelForCausalLLM` from Hugging Face's Auto Classes.

In [ ]:
# Set up the basics
title = "Code Ada ChatBot"
description = "Trained on Microsoft's DialoGPT"

# Instantiate tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

## Define the `user` and `bot` functions

These functions will define how the bot will behave and interact with the user.

In [ ]:
def user(message, history):
    '''
    - Append the user's input to the history.
    '''
    return "", history + [[message, None]]

def bot(history):
    '''
    - Encode (tokenize) the input + end of sentence token.
    - Return the tensor as a pytorch object (required for history).
    '''
    user_message = history[-1][0]
    new_user_input_ids = tokenizer.encode(user_message + tokenizer.eos_token, return_tensors="pt")

    '''
    - Add context by appending the newly inputted tokens to the chat history.
    '''
    bot_input_ids = torch.cat([torch.LongTensor([]), new_user_input_ids], dim=-1)

    '''
    - Generate a response.
    - Make sure to pad using the end of sentence token.
    '''
    response = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).tolist()

    '''
    - Convert the tokens to text.
    - Take the first generated response.
    - Split the responses into lines based on the special <|endoftext|> token.
    '''
    response = tokenizer.decode(response[0]).split("<|endoftext|>")

    '''
    - Convert the response to tuples of lists.
    - The gradio interface requires the wrapping function (in this case, predict)
      to return a single value or a tuple of values,
      with each element in the tuple corresponding to one output component.
    '''
    response = [(response[i], response[i+1]) for i in range(0, len(response) - 1, 2)]

    '''
    - Add the newly generated response to the history.
    '''
    history[-1] = response[0]

    return history


## See the chatbot in action

Launch the gradio interface using `gr.Interface`.

In [ ]:
'''
- Set the properties and launch the gradio interface.
'''

with gr.Blocks() as demo:
#   define variables
  chatbot = gr.Chatbot()

  msg = gr.Textbox()


  clear = gr.Button("Clear")

#   add submit behavior
  msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
      bot, chatbot, chatbot
  )

#   add clear behavior
  clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce2afd835f71e841c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
